# 간경화에 관한 EDA

### 변수 설명
- ID                 : 환자 ID
- N_Days             : 등록 시작 시점부터 censored 까지 기간
- Status             : 
- Drug               : 약물 처리군 / 플라시보 군
- Age                : 연령
- Sex                : 성별
- Ascites            : Y/N 배에 복수가 찼는지
- Hepatomegaly       : Y/N 간 비대칭이 생겼는지
- Spiders            : Y/N 거미혈관증 유무 주로 여성에게 많이 발생할 수 있음 ( 에스트로겐과 관련 )
- Edema              : Y/N 부종 유무
- Bilirubin          : 적혈구 파괴 수치 (파괴 수치가 높을수록 간염이 존재할 확률이 높음)
- Cholesterol        : 콜레스테롤 상승시 지방간이 생김 ( 높을수록 간에 안좋음 )
- Albumin            : 단백질의 한 종류 (값이 하락할 수록 간 기능이 안좋음)
- Copper             : 신체의 필수 미네랄 하나 (단백질, 효소) 수치가 높을 수록 대사 장애 발생 가능성 높음
- Alk_Phos           : (alp) 수치가 높으면 간의 문제가 있을 가능성 높음
- SGOT               : (AST) 간기능 검사의 주요 요소 [0, 40]이 정상 범위
- Tryglicerides      : 중성지방 150 미만이 정상.
- Platelets          : 혈소판 수치가 떨어질 수록 간 경변이 발생 가능
- Prothrombin        : 혈액이 얼마나 빨리 굳는지 측정 시간 ( 시간이 증가하면 응고 시간이 늘어나기 때문에 간경변 의심할 수 있음, 높을수록 의심)
- Stage              : 간경변의 4단계 1부터 4까지의 순서

In [64]:
# load modules
# basic
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools

## graph
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## sklearn
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

## statistics
from scipy.stats import pearsonr
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import bartlett
from scipy.stats import levene
from scipy.stats import chi2_contingency

In [3]:
# load data
data = pd.read_csv('../data/cirrhosis.csv', encoding='utf-8')

In [4]:
# 데이터 결측치 현황
data.isna().sum()

ID                 0
N_Days             0
Status             0
Drug             106
Age                0
Sex                0
Ascites          106
Hepatomegaly     106
Spiders          106
Edema              0
Bilirubin          0
Cholesterol      134
Albumin            0
Copper           108
Alk_Phos         106
SGOT             106
Tryglicerides    136
Platelets         11
Prothrombin        2
Stage              6
dtype: int64

In [5]:
# 데이터 분포 현황 (연속형 데이터만 나옴)
data.describe()

,ID,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,418.000000,418.000000,418.000000,418.000000,284.000000,418.000000,310.000000,312.000000,312.000000,282.000000,407.000000,416.000000,412.000000
mean,209.500000,1917.782297,18533.351675,3.220813,369.510563,3.497440,97.648387,1982.655769,122.556346,124.702128,257.024570,10.731731,3.024272
std,120.810458,1104.672992,3815.845055,4.407506,231.944545,0.424972,85.613920,2140.388824,56.699525,65.148639,98.325585,1.022000,0.882042
min,1.000000,41.000000,9598.000000,0.300000,120.000000,1.960000,4.000000,289.000000,26.350000,33.000000,62.000000,9.000000,1.000000
25%,105.250000,1092.750000,15644.500000,0.800000,249.500000,3.242500,41.250000,871.500000,80.600000,84.250000,188.500000,10.000000,2.000000
50%,209.500000,1730.000000,18628.000000,1.400000,309.500000,3.530000,73.000000,1259.000000,114.700000,108.000000,251.000000,10.600000,3.000000
75%,313.750000,2613.500000,21272.500000,3.400000,400.000000,3.770000,123.000000,1980.000000,151.900000,151.000000,318.000000,11.100000,4.000000
max,418.000000,4795.000000,28650.000000,28.000000,1775.000000,4.640000,588.000000,13862.400000,457.250000,598.000000,721.000000,18.000000,4.000000


In [6]:
# 데이터 속성 확인
data.dtypes

ID                 int64
N_Days             int64
Status            object
Drug              object
Age                int64
Sex               object
Ascites           object
Hepatomegaly      object
Spiders           object
Edema             object
Bilirubin        float64
Cholesterol      float64
Albumin          float64
Copper           float64
Alk_Phos         float64
SGOT             float64
Tryglicerides    float64
Platelets        float64
Prothrombin      float64
Stage            float64
dtype: object

In [7]:
# Stage 변수 확인
data['Stage'].unique()

array([ 4.,  3.,  2.,  1., nan])

In [8]:
# Stage가 NA인 경우를 제외
data = data[~data['Stage'].isna()]

In [9]:
# 없어졌는지 확인
data['Stage'].unique()

array([4., 3., 2., 1.])

In [10]:
# Stage object로 변경
#data['Stage'] = data['Stage'].astype('float')
data['Stage'] = data['Stage'].astype('int64')
data['Stage'] = data['Stage'].astype(str)

## 데이터 다루기

#### 1. Age

- day단위로 되어있기 때문에 365로 나누어 계산해주기
- 소수점은 버리는 것으로 통일

In [11]:
# 단위 맞추기
data['Age_year'] = (data['Age']//365.25).astype(int)

In [12]:
# boxplot of age
fig = px.box(data, x="Age_year")
fig.update_layout(title_text="전체적인 연령 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [13]:
# histogram of age
fig = px.histogram(data, x="Age_year")
fig.update_layout(title_text="전체적인 연령 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

- 전체적으로 대칭형 구조를 형성하고 있음
- 주로 52 - 53세의 환자가 많은 편

In [14]:
# boxplot of age
fig = px.box(data, x="Age_year", color = 'Status')
fig.update_layout(title_text="Status에 따른 연령 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [15]:
# histogram of age and Status
fig = px.histogram(data, x="Age_year", color = 'Status')
fig.update_layout(title_text="Status에 따른 연령 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  barmode = 'overlay')
fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

- 간경화 수술을 받는 나이가 젊은 쪽임을 확인할 수 있음
- Death로 Censored 되는 경우가 연령층이 높은 쪽에서 발생

In [16]:
# boxplot of age
fig = px.box(data, x="Age_year", color = 'Sex')
fig.update_layout(title_text="성별에 따른 연령 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [17]:
# histogram of age and sex
fig = px.histogram(data, x="Age_year", color = 'Sex')
fig.update_layout(title_text="성별에 따른 연령 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  barmode = 'overlay')
fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

- 성별이 여성인 경우가 남성인 경우보다 월등히 많음
- 남성인 경우가 부족할 경우도 있을거 같아 맞추기 위해 oversampling방법인  SMOTE 방법을 활용하면 좋을것 같다는 생각
- 전체적인 분포 또한 대칭과 근사한 구조
- 나이대는 남성이 여성보다 높게 측정된 것으로 보임

In [18]:
# boxplot of age
fig = px.box(data, x="Age_year", color = 'Stage')
fig.update_layout(title_text="Stage에 따른 연령 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [19]:
# histogram of age and stages
fig = px.histogram(data, x="Age_year", color = 'Stage')
fig.update_layout(title_text="Stage에 따른 연령 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  barmode = 'overlay')
fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

- 대체적으로 중앙값이 stages가 커질수록 큰 값을 가지는 형태
- Stages에서 1stage의 표본이 다른 stage보다 적은 상태

#### 2. Ndays

- 진단을 받아 등록된 시점부터 각종 Censord의 사유로 멈추는 시점까지의 기간
- 각 사유별 기간이나 Stage와의 차이를 확인해볼 필요가 있어 보임

In [20]:
# boxplot of age
fig = px.box(data, x="N_Days")
fig.update_layout(title_text="전체적인 관찰기간 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [21]:
# histogram of age
fig = px.histogram(data, x="N_Days")
fig.update_layout(title_text="전체적인 관찰기간 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

- 전체적으로 좌측 쏠림 구조를 형성하고 있음
- 주로 1000 - 2600일의 환자가 많은 편

In [22]:
# boxplot of age
fig = px.box(data, x="N_Days", color = 'Status')
fig.update_layout(title_text="Status에 따른 관찰기간 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [23]:
# histogram of age and Status
fig = px.histogram(data, x="N_Days", color = 'Status')
fig.update_layout(title_text="Status에 따른 관찰기간 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  barmode = 'overlay')
fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

- 확실히 사망이 Censored의 원인인 경우에 관찰 기간이 짧은 것을 확인할 수 있었음
- 주로 좌측으로 분포가 형성되어 있는 것을 확인할 수 있었음

In [24]:
# boxplot of age
fig = px.box(data, x="N_Days", color = 'Sex')
fig.update_layout(title_text="성별에 따른 관찰기간 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [25]:
# histogram of age and sex
fig = px.histogram(data, x="N_Days", color = 'Sex')
fig.update_layout(title_text="성별에 따른 관찰기간 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  barmode = 'overlay')
fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

- 성별에 관계없이 관찰기간은 비슷한 것으로 보임

In [26]:
# boxplot of age
fig = px.box(data, x="N_Days", color = 'Stage')
fig.update_layout(title_text="Stage에 따른 관찰기간 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [27]:
# histogram of age and stages
fig = px.histogram(data, x="N_Days", color = 'Stage')
fig.update_layout(title_text="Stage에 따른 관찰기간 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  barmode = 'overlay')
fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

- stage가 낮을수록 관찰기간이 높음
- stage와 status의 연관성이 있을 수 있음

## 3. Copper

- 정상범위 70 - 130

In [28]:
# boxplot of age
fig = px.box(data, x="Copper")
fig.update_layout(title_text="전체적인 구리 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [29]:
# histogram of age
fig = px.histogram(data, x="Copper")
fig.update_layout(title_text="전체적인 구리 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

- 전체적으로 좌측으로 기울어 있는 모습
- 카이제곱 분포 모양과 비슷함
- 이상치가 종종 발견되는 모습을 보임

In [30]:
# boxplot of age
fig = px.box(data, x="Copper", color = 'Status')
fig.update_layout(title_text="Status에 따른 구리 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [31]:
# histogram of age and Status
fig = px.histogram(data, x="Copper", color = 'Status')
fig.update_layout(title_text="Status에 따른 구리 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  barmode = 'overlay')
fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

- 확실히 사망이 Censored의 원인인 경우에 관찰 기간이 짧은 것을 확인할 수 있었음
- 주로 좌측으로 분포가 형성되어 있는 것을 확인할 수 있었음
- 집단간 차이가 있음

In [32]:
# boxplot of age
fig = px.box(data, x="Copper", color = 'Sex')
fig.update_layout(title_text="성별에 따른 구리 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [33]:
# histogram of age and sex
fig = px.histogram(data, x="Copper", color = 'Sex')
fig.update_layout(title_text="성별에 따른 구리 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  barmode = 'overlay')
fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

- 남성이 여성에 비해 월등히 적지만 분포간의 차이가 존재하는 것으로보임
- 하지만 표본이 적기 때문에 성별간의 차이 유무를 판단하긴 이른것 같아 보임

In [34]:
# boxplot of age
fig = px.box(data, x="Copper", color = 'Stage')
fig.update_layout(title_text="Stage에 따른 구리 boxplot",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [35]:
# histogram of age and stages
fig = px.histogram(data, x="Copper", color = 'Stage')
fig.update_layout(title_text="Stage에 따른 구리 histogram",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  barmode = 'overlay')
fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

- 최빈값이 stage가 커질수록 오른쪽으로 가는 경향이 있음
- 결측 대치를 하기 위해 확인해보면 좋을 요인으로 보임

## 4. 각 변수의 통계량 확인

- 반응변수 Stage를 두고 각 집단간의 차이가 있는지 확인
- status, sex요인은 불균형이 너무 심해서 좋은 결과가 안나올 가능성이 있다.

#### (1) N_days

In [36]:
# Extract data for different 'Status' categories
lv_1 = data['N_Days'][data['Stage'] == '1']
lv_2 = data['N_Days'][data['Stage'] == '2']
lv_3 = data['N_Days'][data['Stage'] == '3']
lv_4 = data['N_Days'][data['Stage'] == '4']

# Perform ANOVA
f_statistic, p_value = stats.f_oneway(lv_1, lv_2, lv_3, lv_4)
print(f'ANOVA F-statistic: {f_statistic:.2f}')
print(f'ANOVA p-value: {p_value:.4f}')

# Perform Tukey's test for pairwise comparisons
tukey_results = pairwise_tukeyhsd(data['N_Days'], data['Stage'])
print(tukey_results)

ANOVA F-statistic: 21.74
ANOVA p-value: 0.0000
    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff  p-adj    lower      upper   reject
-----------------------------------------------------------
     1      2  -264.9726 0.7087  -904.4909  374.5457  False
     1      3  -657.3579 0.0308  -1272.249  -42.4669   True
     1      4 -1234.5595    0.0 -1852.2464 -616.8726   True
     2      3  -392.3853 0.0199  -740.4068  -44.3639   True
     2      4   -969.587    0.0 -1322.5248 -616.6492   True
     3      4  -577.2016    0.0  -883.2611 -271.1421   True
-----------------------------------------------------------


- one-way ANOVA결과 집단간의 차이가 존재하는 것으로 확인 되었음
- 일반적으로 사용하는 tukey 검정을 통해 집단간의 구체적인 차이를 확인
- Stage 1, 2의 N_days는 차이가 없고 나머지 집단과는 차이가 존재하는것을 확인할 수 있었음

#### (2) Age_year

In [37]:
# Extract data for different 'Status' categories
lv_1 = data['Age_year'][data['Stage'] == '1']
lv_2 = data['Age_year'][data['Stage'] == '2']
lv_3 = data['Age_year'][data['Stage'] == '3']
lv_4 = data['Age_year'][data['Stage'] == '4']

# Perform ANOVA
f_statistic, p_value = stats.f_oneway(lv_1, lv_2, lv_3, lv_4)
print(f'ANOVA F-statistic: {f_statistic:.2f}')
print(f'ANOVA p-value: {p_value:.4f}')

# Perform Tukey's test for pairwise comparisons
tukey_results = pairwise_tukeyhsd(data['Age_year'], data['Stage'])
print(tukey_results)

ANOVA F-statistic: 6.98
ANOVA p-value: 0.0001
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     1      2    2.721 0.6923 -3.6849  9.1269  False
     1      3   2.2086 0.7915 -3.9506  8.3678  False
     1      4   6.9236 0.0213  0.7364 13.1108   True
     2      3  -0.5124 0.9814 -3.9984  2.9736  False
     2      4   4.2026 0.0123  0.6673  7.7379   True
     3      4    4.715 0.0005  1.6493  7.7807   True
----------------------------------------------------


- 연령이 Stage간의 차이가 존재하는 것을 확인할 수 있음
- Stage가 4인 집단의 연령대와 나머지 집단의 연령대의 차이를 확인할 수 있었음

#### (3) Copper

In [38]:
# extract data with not null values
data2 = data[~data['Copper'].isna()]

# Extract data for different 'Status' categories
lv_1 = data2['Copper'][(data2['Stage'] == '1')]
lv_2 = data2['Copper'][(data2['Stage'] == '2')]
lv_3 = data2['Copper'][(data2['Stage'] == '3')]
lv_4 = data2['Copper'][(data2['Stage'] == '4')]

# Perform ANOVA
f_statistic, p_value = stats.f_oneway(lv_1, lv_2, lv_3, lv_4)
print(f'ANOVA F-statistic: {f_statistic:.2f}')
print(f'ANOVA p-value: {p_value:.4f}')

# Perform Tukey's test for pairwise comparisons
tukey_results = pairwise_tukeyhsd(data2['Copper'], data2['Stage'])
print(tukey_results)

ANOVA F-statistic: 8.53
ANOVA p-value: 0.0000
 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     1      2   5.2178 0.9959 -54.2747  64.7103  False
     1      3  29.2625 0.5443 -27.5582  86.0832  False
     1      4  64.2894 0.0205   7.0985 121.4802   True
     2      3  24.0447 0.2308  -8.6729  56.7623  False
     2      4  59.0715    0.0  25.7152  92.4279   True
     3      4  35.0269 0.0084   6.7095  63.3442   True
------------------------------------------------------


In [39]:
# extract data with not null values
data2 = data[~data['Copper'].isna()]

# Extract data for different 'Status' categories
lv_D = data2['Copper'][(data2['Status'] == 'D')]
lv_C = data2['Copper'][(data2['Status'] == 'C')]
lv_CL = data2['Copper'][(data2['Status'] == 'CL')]


# Perform ANOVA
f_statistic, p_value = stats.f_oneway(lv_1, lv_2, lv_3, lv_4)
print(f'ANOVA F-statistic: {f_statistic:.2f}')
print(f'ANOVA p-value: {p_value:.4f}')

# Perform Tukey's test for pairwise comparisons
tukey_results = pairwise_tukeyhsd(data2['Copper'], data2['Status'])
print(tukey_results)

ANOVA F-statistic: 8.53
ANOVA p-value: 0.0000
 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     C     CL  57.3892 0.0081  12.3658 102.4127   True
     C      D  68.8005    0.0  46.7563  90.8447   True
    CL      D  11.4113 0.8274 -34.4026  57.2252  False
------------------------------------------------------


- Age와 마찬가지로 4 집단과의 차이만 유의한것을 확인할 수 있었음
- 따라서 결측치 대치는 Stage의 4집단과 나머지 집단을 구분을 두고 진행하면 좋을것같아 보임
- status는 C와 그 외 나머지를 구분할 수 있을것 같음

## 5. 데이터 이상치 확인
- Copper의 이상치가 유독 눈에 띄눈 위치에 존재함
- 결측치는 데이터에 따라 수정해야 하는 이상치와 동시에 변경하면 좋을것 같음

- 고민되는 방법 
    - 1. Cook's 거리 (영향이 있는 점인지 아닌지 통계적 판단이 가미 된 값) -> 일단 PASS
    - 2. 해당 집단의 다른 변수들 분포와 값을 비교하는 것

In [40]:
# Copper 이상치 골라내기
data['coppergroup'] = "0"
data['coppergroup'] [data['Copper'] > 243] = "1"

C:\Users\abc\AppData\Local\Temp\ipykernel_13636\2331803364.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
# copper graphs

## 'Stage'로 색 구분하고 'x' 변수의 count 그래프 그리기
for i in data.columns[1:-1]:
    fig = px.histogram(data, y=i, color='coppergroup')
    fig.update_layout(title=f'Stage별 {i}의 Count 그래프',
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  barmode = 'group',
                  width=800,
                  height=400
                  )
    fig.update_traces(#marker_color= 히스토그램 색, 
                    #marker_line_width=히스토그램 테두리 두깨,                            
                    #marker_line_color=히스토그램 테두리 색,
                    marker_opacity = 0.4,
                    )
    ## 그래프 보이기
    fig.show()

In [42]:
data[data['Copper'] == 4]

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,...,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Age_year,coppergroup
160,161,2797,C,Placebo,15612,F,N,N,N,N,...,3.56,4.0,964.0,120.9,180.0,269.0,9.6,2,42,0


In [43]:
data[(data['coppergroup'] == '1') & (data['Hepatomegaly'] != 'N')]

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,...,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Age_year,coppergroup
17,18,131,D,D-penicillamine,19698,F,N,Y,Y,Y,...,2.80,588.0,961.0,280.55,200.0,283.0,12.4,4,53,1
22,23,264,D,Placebo,20442,F,Y,Y,Y,Y,...,2.94,558.0,6064.8,227.04,191.0,214.0,11.7,4,55,1
53,54,1434,D,D-penicillamine,14317,F,Y,Y,Y,Y,...,3.40,262.0,5487.2,73.53,125.0,254.0,11.0,4,39,1
73,74,1827,D,D-penicillamine,18964,F,N,Y,Y,N,...,3.99,280.0,967.0,89.90,309.0,278.0,11.0,4,51,1
79,80,890,D,Placebo,24622,M,N,Y,N,N,...,3.72,269.0,1303.0,176.70,91.0,360.0,11.2,4,67,1
137,138,1297,D,D-penicillamine,18719,M,N,Y,N,N,...,3.93,262.0,2424.0,145.70,218.0,252.0,10.5,3,51,1
147,148,1427,D,Placebo,11273,F,N,Y,N,N,...,3.26,247.0,3836.0,198.40,280.0,330.0,9.8,3,30,1
148,149,762,D,D-penicillamine,22574,M,N,Y,Y,S,...,3.79,290.0,1664.0,102.30,112.0,140.0,9.9,4,61,1
183,184,974,D,Placebo,13736,F,N,Y,N,N,...,3.55,358.0,2412.0,167.40,140.0,471.0,9.8,3,37,1
186,187,733,D,Placebo,13073,F,N,Y,N,N,...,3.43,251.0,2870.0,153.45,137.0,268.0,11.5,3,35,1


In [44]:
data[(data['coppergroup'] == '1') & (data['Hepatomegaly'] == 'N')]

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,...,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Age_year,coppergroup
21,22,673,D,D-penicillamine,20555,F,N,N,Y,N,...,3.63,464.0,1376.0,120.90,55.0,173.0,11.6,4,56,1
47,48,4427,C,Placebo,17947,M,N,N,N,N,...,3.70,281.0,10396.8,188.34,178.0,214.0,11.0,3,49,1
119,120,2033,CL,D-penicillamine,12839,M,N,N,N,N,...,3.98,444.0,766.0,130.20,210.0,344.0,10.6,3,35,1
192,193,797,D,Placebo,20736,F,N,N,N,N,...,3.19,267.0,2184.0,161.20,157.0,382.0,10.4,4,56,1


### 6. 범주형 변수 chisquare-test

In [68]:
# 범주형 변수만 가져오기
data_categorical = data[list(data.dtypes[data.dtypes == 'object'].index[1:])]
data_categorical.columns

Index(['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Stage',
       'coppergroup'],
      dtype='object')

In [74]:
for i in data_categorical.columns:
    if i != 'Stage':
        print(f'{i} 변수 결과')
        tmp = pd.crosstab(data['Stage'], data[i])
        chi2, p, dof, expected = chi2_contingency(tmp)
        msg = 'Test Statistic: {}\np-value: {}\nDegree of Freedom: {}'
        print(msg.format(round(chi2, 2), round(p, 4), dof))
        print(expected)

Drug 변수 결과
Test Statistic: 4.63
p-value: 0.2013
Degree of Freedom: 3
[[ 8.1025641   7.8974359 ]
 [33.92948718 33.07051282]
 [60.76923077 59.23076923]
 [55.19871795 53.80128205]]
Sex 변수 결과
Test Statistic: 0.88
p-value: 0.8307
Degree of Freedom: 3
[[ 18.75728155   2.24271845]
 [ 82.17475728   9.82524272]
 [138.44660194  16.55339806]
 [128.62135922  15.37864078]]
Ascites 변수 결과
Test Statistic: 31.94
p-value: 0.0
Degree of Freedom: 3
[[ 14.76923077   1.23076923]
 [ 61.84615385   5.15384615]
 [110.76923077   9.23076923]
 [100.61538462   8.38461538]]
Hepatomegaly 변수 결과
Test Statistic: 71.21
p-value: 0.0
Degree of Freedom: 3
[[ 7.79487179  8.20512821]
 [32.64102564 34.35897436]
 [58.46153846 61.53846154]
 [53.1025641  55.8974359 ]]
Spiders 변수 결과
Test Statistic: 27.99
p-value: 0.0
Degree of Freedom: 3
[[11.38461538  4.61538462]
 [47.67307692 19.32692308]
 [85.38461538 34.61538462]
 [77.55769231 31.44230769]]
Edema 변수 결과
Test Statistic: 30.44
p-value: 0.0
Degree of Freedom: 6
[[ 17.73786408   2.

In [73]:
data.columns

Index(['ID', 'N_Days', 'Status', 'Drug', 'Age', 'Sex', 'Ascites',
       'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol',
       'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets',
       'Prothrombin', 'Stage', 'Age_year', 'coppergroup'],
      dtype='object')

In [75]:
print('Status 변수 결과')
tmp = pd.crosstab(data['Stage'], data['Status'])
chi2, p, dof, expected = chi2_contingency(tmp)
msg = 'Test Statistic: {}\np-value: {}\nDegree of Freedom: {}'
print(msg.format(round(chi2, 2), round(p, 4), dof))
print(expected)

Status 변수 결과
Test Statistic: 48.12
p-value: 0.0
Degree of Freedom: 6
[[11.72330097  1.27427184  8.00242718]
 [51.3592233   5.58252427 35.05825243]
 [86.52912621  9.40533981 59.06553398]
 [80.38834951  8.73786408 54.87378641]]


In [77]:
# extract data with not null values
data2 = data[~data['Tryglicerides'].isna()]

# Extract data for different 'Status' categories
lv_1 = data2['Tryglicerides'][(data2['Stage'] == '1')]
lv_2 = data2['Tryglicerides'][(data2['Stage'] == '2')]
lv_3 = data2['Tryglicerides'][(data2['Stage'] == '3')]
lv_4 = data2['Tryglicerides'][(data2['Stage'] == '4')]

# Perform ANOVA
f_statistic, p_value = stats.f_oneway(lv_1, lv_2, lv_3, lv_4)
print(f'ANOVA F-statistic: {f_statistic:.2f}')
print(f'ANOVA p-value: {p_value:.4f}')

# Perform Tukey's test for pairwise comparisons
tukey_results = pairwise_tukeyhsd(data2['Tryglicerides'], data2['Stage'])
print(tukey_results)

ANOVA F-statistic: 2.22
ANOVA p-value: 0.0864
 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     1      2  23.1047 0.6473 -28.0023 74.2117  False
     1      3  39.8884 0.1543  -9.1093  88.886  False
     1      4   38.217 0.1917 -11.2573 87.6913  False
     2      3  16.7837 0.3623  -9.7973 43.3647  False
     2      4  15.1123 0.4861 -12.3373  42.562  False
     3      4  -1.6714 0.9977 -24.9593 21.6166  False
-----------------------------------------------------


In [80]:
# extract data with not null values
data2 = data[~data['Prothrombin'].isna()]

# Extract data for different 'Status' categories
lv_1 = data2['Prothrombin'][data2['Stage'] == '1']
lv_2 = data2['Prothrombin'][data2['Stage'] == '2']
lv_3 = data2['Prothrombin'][data2['Stage'] == '3']
lv_4 = data2['Prothrombin'][data2['Stage'] == '4']

# Perform ANOVA
f_statistic, p_value = stats.f_oneway(lv_1, lv_2, lv_3, lv_4)
print(f'ANOVA F-statistic: {f_statistic:.2f}')
print(f'ANOVA p-value: {p_value:.4f}')

# Perform Tukey's test for pairwise comparisons
tukey_results = pairwise_tukeyhsd(data2['Prothrombin'], data2['Stage'])
print(tukey_results)

ANOVA F-statistic: 13.62
ANOVA p-value: 0.0000
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2   -0.255 0.7186 -0.8801 0.3701  False
     1      3   -0.311 0.5426 -0.9129  0.291  False
     1      4   0.3676 0.3984 -0.2373 0.9724  False
     2      3   -0.056 0.9728 -0.3894 0.2775  False
     2      4   0.6226    0.0  0.2839 0.9612   True
     3      4   0.6785    0.0  0.3847 0.9723   True
---------------------------------------------------


### 7. demographic data

In [114]:
# 연속형 변수만 가져오기
data_continuous = data[list(data.dtypes[data.dtypes != 'object'].index)]
data_continuous.columns

Index(['ID', 'N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Age_year'],
      dtype='object')

In [115]:
# 범주형 변수만 가져오기
data_categorical = data[list(data.dtypes[data.dtypes == 'object'].index)]
data_categorical.columns

Index(['Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema',
       'Stage', 'coppergroup'],
      dtype='object')

In [116]:
# 기준점인 stage 추가
data_continuous['Stage'] = data['Stage']
data_continuous.columns

C:\Users\abc\AppData\Local\Temp\ipykernel_13636\78443904.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['ID', 'N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Age_year', 'Stage'],
      dtype='object')

In [117]:
# 연속형 변수 중앙값(표준오차)
data_continuous.groupby('Stage').median()

,ID,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Age_year
Stage,,,,,,,,,,,,,
1,174.0,2644.0,16929.0,0.80,239.0,3.770,64.0,706.0,64.325,84.0,270.5,10.15,46.0
2,209.5,2409.5,17897.0,0.95,298.0,3.615,49.5,1164.0,108.500,101.0,277.0,10.40,48.0
3,211.0,1810.0,17947.0,1.30,324.0,3.610,67.5,1257.5,112.375,119.0,252.0,10.40,49.0
4,204.0,1207.0,19724.0,2.55,299.0,3.340,98.5,1428.0,122.450,106.0,216.0,11.00,54.0


In [119]:
round(data_continuous.groupby('Stage').std(), 2)

,ID,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Age_year
Stage,,,,,,,,,,,,,
1,121.54,1092.81,3486.56,1.79,120.78,0.35,47.09,1895.16,42.96,34.78,85.06,1.68,9.56
2,112.04,1069.09,3515.28,4.14,185.58,0.38,53.80,2142.81,62.73,54.45,101.35,1.08,9.66
3,122.17,973.84,3695.81,4.19,292.01,0.38,75.00,2353.81,54.97,57.71,83.51,0.76,10.11
4,125.54,1040.37,3953.22,4.86,171.65,0.44,105.87,1934.23,54.35,79.85,97.61,1.00,10.89


In [100]:
# 범주변수 count
for i in data_categorical.columns:
    if i != 'Stage':
        print(f'{i} 변수 결과')
        print(pd.crosstab(data[i], data['Stage']))

Status 변수 결과
Stage    1   2   3   4
Status                
C       19  64  97  50
CL       0   5  10  10
D        2  23  48  84
Drug 변수 결과
Stage             1   2   3   4
Drug                           
D-penicillamine  12  35  56  55
Placebo           4  32  64  54
Sex 변수 결과
Stage   1   2    3    4
Sex                    
F      18  84  139  127
M       3   8   16   17
Ascites 변수 결과
Stage     1   2    3   4
Ascites                 
N        16  65  119  88
Y         0   2    1  21
Hepatomegaly 변수 결과
Stage          1   2   3   4
Hepatomegaly                
N             16  48  67  21
Y              0  19  53  88
Spiders 변수 결과
Stage     1   2   3   4
Spiders                
N        15  58  90  59
Y         1   9  30  50
Edema 변수 결과
Stage   1   2    3    4
Edema                  
N      20  86  138  104
S       1   5   14   24
Y       0   1    3   16
coppergroup 변수 결과
Stage         1   2    3    4
coppergroup                  
0            21  92  149  133
1             0   0    6   1

## Apendix A - 결측값으로 남은 카테고리 데이터의 비율 계산

In [45]:
# Stage 확인
data['Stage'].value_counts() / data['Stage'].count()

Stage
3    0.376214
4    0.349515
2    0.223301
1    0.050971
Name: count, dtype: float64

In [46]:
# 성비 확인
data['Sex'].value_counts() / data['Sex'].count()

Sex
F    0.893204
M    0.106796
Name: count, dtype: float64

In [47]:
# 결측치 제거한 성비 확인
drug_data = data[~data['Drug'].isna()]
drug_data['Sex'].value_counts() / drug_data['Sex'].count()

Sex
F    0.884615
M    0.115385
Name: count, dtype: float64

- 전체적인 성비율과 제공하지 않은 인구의 성비가 거의 같은것으로 보아 자연스러운 층화추출로 볼 수도 있음